Install the necessary libraries

In [ ]:
!pip install llama-index
!pip install openai
!pip install langchain
!pip install sentence-transformers
!pip install llama-index-llms-azure-openai
!pip install llama-index-embeddings-langchain
!pip install llama-index-embeddings-openai

In [ ]:
import openai

openai.api_key= "key here"


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="HuggingFaceH4/zephyr-7b-gemma-v0.1") # fast model with competitive perfomance
#all-MiniLM-L6-v2

#Use your own documents

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.azure_openai import AzureOpenAI



service_context= ServiceContext.from_defaults(embed_model=embed_model)

documents= SimpleDirectoryReader('/content/gdrive/MyDrive/original_data').load_data()
index= VectorStoreIndex.from_documents(documents, service_context=service_context)


In [ ]:
query_engine= index.as_query_engine()
response= query_engine.query("Rewrite the text more elegantly in the shakespear style?")
print(response)


In [ ]:
from llama_index.core import Document


documents1= documents + [Document(text=str(response))]

index1 = VectorStoreIndex.from_documents(documents1,service_context=service_context)

query_engine1= index1.as_query_engine()
response1= query_engine1.query("prompt rewrite the text in dr.seuss style?")
print(response1)


In [ ]:

from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(embed_model='sentence-transformers/sentence-t5-base')

search_text= "rewrite the text in dr.seuss style?"
embedding1= embed_model.get_text_embedding(search_text)

search_text= "rewrite the text in childish style?"
embedding2= embed_model.get_text_embedding(search_text)


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer("sentence-t5-base")

# e.g.) base_text="Improve this text:", compare_text="Improve the text:"
base_embedding = model.encode(str(embedding1))
compare_embedding = model.encode(str(embedding2))

cos_sim = cosine_similarity([base_embedding], [compare_embedding])

exponent = 3
score = cos_sim.item() ** exponent
print(score)